Initial attempt to get metadata for mapillary using overpass api, ended up using geopy's api to overpass to get results. This notebook is probably no longer useful

In [30]:
import requests
import json
from tqdm import tqdm
import os
import overpy
api = overpy.Overpass()
api.default_max_retry_count = 5
api.default_retry_timeout = 10
# https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0

In [4]:
overpass_url = "http://overpass-api.de/api/interpreter"
countries = [("US","4","State"), ("CN","4","Province"), ("IN","4","State"),("RU","3","Federal districts"), ("FR","4","Regions")]
country_coords = {c[0]:{"admin_level":c[1],"division_name":c[2],"files":dict()} for c in countries}
admin = "admin_level"
for file in os.listdir("../country211/train"):
    if file in country_coords.keys():
        for file2 in os.listdir("../country211/train/"+file):
            country_coords[file]["files"][file2] = dict()
            id_, lat, long = file2.split("_")
            country_coords[file]["files"][file2]["lat"] = lat
            country_coords[file]["files"][file2]["long"] = long[:-4]
            country_coords[file]["files"][file2]["id"] = id_
            

In [142]:
# country_coords["US"]
# list(country_coords["US"]["files"])

In [ ]:

result_list = []
for country in ["US","CN"]:
    query = f""
    country_files = list(country_coords[country]["files"])
    for idx, file in tqdm(enumerate(country_files)):
        lat = country_coords[country]["files"][file]["lat"]
        long = country_coords[country]["files"][file]["long"]
        admin = country_coords[country]["admin_level"]
        query = f"[out:csv];" + (f"is_in({lat},{long});area._[admin_level={admin}];out;")
        
        try:
            result = api.query(query)
            result_list.append(result) 
        except:
            print(idx)


3it [00:00, 20.75it/s]

0
1
2
3
4

6it [00:00, 20.28it/s]


5
6
7


11it [00:00, 18.32it/s]

...

98
99
100
101


105it [00:05, 17.67it/s]

102
103
104
105


109it [00:05, 17.53it/s]

106
107
108
109

In [28]:
result_list

In [24]:
for area in result.areas:
    print(area.tags["name:en"])

Maryland
Washington
California
Mississippi
Texas
Virginia
New York
Nevada
Maine
Oregon
Pennsylvania
New Jersey
Connecticut
Florida
District of Columbia
Arizona
Ohio
South Carolina
Indiana
Tennessee
Illinois
Iowa
Wisconsin
Hawaii
Massachusetts
Wyoming
Arkansas
Michigan
Alabama
Alaska
North Carolina
Georgia
Kentucky
Rhode Island
Colorado
Minnesota
West Virginia


In [ ]:
data["elements"]

In [6]:
testquery = "[out:json];is_in(39.279399,-76.606498);area._[admin_level=4];out;"
result = api.query(testquery)

In [18]:
result.areas[0].tags["name:en"]

'Maryland'

In [67]:
import pickle
with open('country_coords.pickle', 'wb') as handle:
    pickle.dump(country_coords, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [ ]:
country_coords

example image in china 861748_32.017855_118.794647

example image in India 101127_27.216661_77.950291

example image in France 903539_48.852522_2.348069

In [143]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
is_in(27.216661,77.950291);

out body ;
is_in(48.852522,2.348069);
area._[admin_level=4];
out ;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [147]:

# data["elements"][3]

In [134]:
for element in data["elements"]:
    if element["type"] == "area":
#         print(element)
        try:
            print("admin level ", element["tags"]["admin_level"])
            print("name ", element["tags"]["name"])
        except:
            print(element)

admin level  2
name  India
admin level  4
name  Uttar Pradesh
admin level  5
name  Agra
admin level  6
name  Agra
{'type': 'area', 'id': 3611203285, 'tags': {'multipolygon': 'river_basin', 'name': 'Ganga Yamuna River Basin', 'type': 'multipolygon'}}
admin level  10
name  48
admin level  8
name  Agra Nagar Nigam
admin level  tbd
name  Agra division
admin level  4
name  Île-de-France


In [71]:
data["elements"][0]["tags"]["name:en"]
# data["elements"][0]["tags"]


'Ile-de-France'

In [3]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
data

In [ ]:
1627214_42.510229_1.539802

In [ ]:
<node id="25496583" lat="51.5173639" lon="-0.140043" </node>